# Trabalho prático AC

## Trabalho realizado por:

Margarida Vila-chã, 20210504

Miguel Duarte Silva, 20210504

Sebastião Lessa, 202103238


## Algoritmo escolhido e breve explicação:

O algoritmo escolhido pelo nosso grupo foi o Random Forrest.

Este algoritmo é um algoritmo de aprendizagem computacional supervisionado. Ele usa uma combinação de árvores de decisão aleatórias. O processo começa com a criação de um grande número de árvores (floresta), em que cada uma das árvores criadas é treinada com uma amostra diferente. Isto permite a criação de árvores suficientemente diferentes, o que leva a que elas juntas consigam lidar com uma ampla variedade de situações, reduzindo o risco de overffitting.

Baseamos-nos nos seguintes testes:

- [credit-g](https://www.openml.org/search?type=run&id=591&collections.id=99&run_task.task_id=31&sort=date)
- [spam base](https://www.openml.org/search?type=run&id=6785&collections.id=99&run_task.task_id=43)
- [qsar-biodeg](https://www.openml.org/search?type=run&id=519587&collections.id=99&run_task.task_id=9957)

## Importação das bibliotecas necessárias:

In [6]:
from scipy.io import arff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from RandomForestClassifier import RandomForestClassifier
from RandomForestClassifierUpgraded import RandomForestClassifierUpgraded

## Inicialização dos datasets

In [7]:
credit = pd.DataFrame(arff.loadarff('datasets/credit-g.arff')[0])
spam = pd.DataFrame(arff.loadarff('datasets/spambase.arff')[0])
qsar = pd.DataFrame(arff.loadarff('datasets/qsar-biodeg.arff')[0])

## Teste com o dataset credit-g

## Teste com o dataset spam base

In [8]:
spam

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_%3B,char_freq_%28,char_freq_%5B,char_freq_%21,char_freq_%24,char_freq_%23,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61.0,278.0,b'1'
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101.0,1028.0,b'1'
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485.0,2259.0,b'1'
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40.0,191.0,b'1'
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40.0,191.0,b'1'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3.0,88.0,b'0'
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4.0,14.0,b'0'
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6.0,118.0,b'0'
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5.0,78.0,b'0'


## Teste com o dataset qsar-biodeg